In [42]:
import pandas as pd
import datetime as dt
import urllib
import zipfile
import re
from geopy.distance import vincenty

In [43]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
urllib.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <httplib.HTTPMessage instance at 0x114c05cb0>)

In [44]:
zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()

In [45]:
stop_times= pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

In [46]:
stop_times.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901N,1,NaN,0,0,NaN
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902N,2,NaN,0,0,NaN


In [47]:
stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']

In [48]:
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [49]:
stop_times.loc[stop_times[stop_times['train+direction'].isin(['N..N72R','N..S72R'])].index,['train']]='W'

In [50]:
stop_train= stop_times[['stop_id','train']].drop_duplicates()

In [51]:
stop_train['stop_id'] = map(lambda x:x[:-1],stop_train['stop_id'])

In [52]:
stop_train = stop_train.drop_duplicates().sort_values(['train','stop_id']).reset_index(drop= True)

In [53]:
set(stop_train.train)

{'1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'FS',
 'G',
 'GS',
 'H',
 'J',
 'L',
 'M',
 'N',
 'Q',
 'R',
 'SI',
 'W'}

In [54]:
stops=pd.read_csv('GTFS_nyc_Subway/stops.txt')

In [55]:
stop_train1 = stop_train.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on='stop_id',how='left')

In [56]:
stop_train1.to_csv('static/data/GTFS_nyc_Subway/stop_train.csv', index=False)

In [57]:
stop_train1.head()

,stop_id,train,stop_name,stop_lat,stop_lon
0,101,1,Van Cortlandt Park - 242 St,40.889248,-73.898583
1,103,1,238 St,40.884667,-73.900870
2,104,1,231 St,40.878856,-73.904834
3,106,1,Marble Hill - 225 St,40.874561,-73.909831
4,107,1,215 St,40.869444,-73.915279


In [58]:
stop_times['stop_id'] = stop_times['stop_id'].str[:-1]

In [59]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901,1,A20171105SAT,036000,GS.N01R,GS,SAT
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902,2,A20171105SAT,036000,GS.N01R,GS,SAT
2,A20171105SAT_036400_GS.S01R,06:04:00,06:04:00,902,1,A20171105SAT,036400,GS.S01R,GS,SAT
3,A20171105SAT_036400_GS.S01R,06:05:30,06:05:30,901,2,A20171105SAT,036400,GS.S01R,GS,SAT
4,A20171105SAT_037000_GS.N01R,06:10:00,06:10:00,901,1,A20171105SAT,037000,GS.N01R,GS,SAT


In [60]:
stop_times['train+direction'].unique()

array(['GS.N01R', 'GS.S01R', 'GS.N03R', 'GS.S03R', '1..S03R', '1..N03R',
       '1..S04R', '1..N12R', '1..S12R', '1..N13R', '2..S08R', '2..N08R',
       '2..S01R', '2..N01R', '2..S05R', '2..S06R', '2..S07R', '2..N03R',
       '2..N02R', '2..S03R', '2..S12R', '2..N12R', '3..S42R', '3..N42R',
       '3..S01R', '3..N01R', '3..S03R', '3..S43R', '3..N43R', '4..S01R',
       '4..N13R', '4..S13R', '4..N01R', '4..S34R', '4..N43R', '4..N02R',
       '4..S06R', '4..N06R', '4..S28R', '4..N34R', '4..S39R', '4..S36R',
       '4..S30R', '4..S05R', '4..N64R', '4..N03R', '4..S03R', '4..S10R',
       '5..S32R', '5..N32R', '5..N01R', '5..S04R', '5..N26R', '5..N64R',
       '5..S09R', '5..N71R', '5..S03R', '5..N66R', '5..N74R', '5..S08R',
       '5..N73R', '5..S07R', '5..N68R', '5..S01R', '5..N76R', '5..S02R',
       '5..N72R', '5..S21R', '5..S14R', '5..N69R', '5..S42R', '5..N60R',
       '5..N14R', '5..S12R', '5..N67R', '5..S11R', '5..S15R', '5..N65R',
       '5..S06R', '5..N70R', '5..S13R', '5..N15R', 

In [61]:
stop_sequence = stop_times[['stop_id','train','stop_sequence', 'train+direction']].drop_duplicates().reset_index()
del stop_sequence['index']

In [62]:
stop_train2 = stop_train1.merge(stop_sequence, on=['stop_id', 'train'])

In [63]:
stop_train2 = stop_train2.sort_values(['train+direction', 'stop_sequence'])

In [64]:
stop_train2

,stop_id,train,stop_name,stop_lat,stop_lon,stop_sequence,train+direction
194,142,1,South Ferry,40.702068,-74.013664,1,1..N03R
188,139,1,Rector St,40.707513,-74.013783,2,1..N03R
182,138,1,Cortlandt St,40.711835,-74.012188,3,1..N03R
176,137,1,Chambers St,40.715478,-74.009266,4,1..N03R
170,136,1,Franklin St,40.719318,-74.006886,5,1..N03R
164,135,1,Canal St,40.722854,-74.006277,6,1..N03R
158,134,1,Houston St,40.728251,-74.005367,7,1..N03R
152,133,1,Christopher St - Sheridan Sq,40.733422,-74.002906,8,1..N03R
146,132,1,14 St,40.737826,-74.000201,9,1..N03R
140,131,1,18 St,40.741040,-73.997871,10,1..N03R


In [65]:
stop_train2 = stop_train2.groupby('train').apply(lambda df: df[
    df['train+direction'] == 
    df.groupby('train+direction').apply(lambda x: x['stop_sequence'].max()).idxmax()
]).reset_index(drop=True)
stop_train2.groupby('train').apply(lambda x: x['train+direction'].unique())

train
1     [1..N03R]
2     [2..N08R]
3     [3..N01R]
4     [4..N13R]
5     [5..N15R]
6     [6..N01R]
7     [7..N97R]
A     [A..N09R]
B     [B..N45R]
C     [C..N04R]
D     [D..N05R]
E     [E..N05R]
F     [F..N69R]
FS    [FS.N01R]
G     [G..N13R]
GS    [GS.N01R]
H     [H..N21R]
J     [J..N12R]
L     [L..N01R]
M     [M..N27R]
N     [N..N20R]
Q     [Q..N19R]
R     [R..N93R]
SI    [SI.N03R]
W     [N..N72R]
dtype: object

In [66]:
stop_train2.loc[stop_train2['train'] == 'F']

,stop_id,train,stop_name,stop_lat,stop_lon,stop_sequence,train+direction
496,D43,F,Coney Island - Stillwell Av,40.577422,-73.981233,1,F..N69R
497,D42,F,W 8 St - NY Aquarium,40.576127,-73.975939,2,F..N69R
498,F39,F,Neptune Av,40.581011,-73.974574,3,F..N69R
499,F38,F,Avenue X,40.589620,-73.974250,4,F..N69R
500,F36,F,Avenue U,40.596063,-73.973357,5,F..N69R
501,F35,F,Kings Hwy,40.603217,-73.972361,6,F..N69R
502,F34,F,Avenue P,40.608944,-73.973022,7,F..N69R
503,F33,F,Avenue N,40.615140,-73.974197,8,F..N69R
504,F32,F,Bay Pkwy,40.620769,-73.975264,9,F..N69R
505,F31,F,Avenue I,40.625322,-73.976127,10,F..N69R


In [67]:
stop_train2.to_csv('static/data/GTFS_nyc_Subway/stop_train.csv', index=False)

In [68]:
stop_train['node_id']= stop_train['stop_id'] + '_' + stop_train['train']

In [69]:
stop_train2.head()

,stop_id,train,stop_name,stop_lat,stop_lon,stop_sequence,train+direction
0,142,1,South Ferry,40.702068,-74.013664,1,1..N03R
1,139,1,Rector St,40.707513,-74.013783,2,1..N03R
2,138,1,Cortlandt St,40.711835,-74.012188,3,1..N03R
3,137,1,Chambers St,40.715478,-74.009266,4,1..N03R
4,136,1,Franklin St,40.719318,-74.006886,5,1..N03R


In [70]:
transfer = stop_train2.groupby('stop_name')['train'].apply(lambda x: x.tolist()).rename('lines')
transfer.head()

stop_name
1 Av                                    [L]
103 St                   [1, 4, 6, A, B, C]
103 St - Corona Plaza                   [7]
104 St                                  [J]
110 St                               [4, 6]
Name: lines, dtype: object

In [71]:
transfer.to_csv('static/data/GTFS_nyc_Subway/transfer_train.csv', index=False)

In [72]:
transfer = transfer.to_frame().reset_index()

In [73]:
stop_train2 = stop_train2.merge(transfer, on='stop_name')

In [74]:
stop_train2.head(10)

,stop_id,train,stop_name,stop_lat,stop_lon,stop_sequence,train+direction,lines
0,142,1,South Ferry,40.702068,-74.013664,1,1..N03R,"[1, 5]"
1,142,5,South Ferry,40.702068,-74.013664,1,5..N15R,"[1, 5]"
2,139,1,Rector St,40.707513,-74.013783,2,1..N03R,"[1, 5, N, R, W]"
3,139,5,Rector St,40.707513,-74.013783,2,5..N15R,"[1, 5, N, R, W]"
4,R26,N,Rector St,40.707220,-74.013342,24,N..N20R,"[1, 5, N, R, W]"
5,R26,R,Rector St,40.707220,-74.013342,18,R..N93R,"[1, 5, N, R, W]"
6,R26,W,Rector St,40.707220,-74.013342,2,N..N72R,"[1, 5, N, R, W]"
7,138,1,Cortlandt St,40.711835,-74.012188,3,1..N03R,"[1, 5, N, R, W]"
8,138,5,Cortlandt St,40.711835,-74.012188,3,5..N15R,"[1, 5, N, R, W]"
9,R25,N,Cortlandt St,40.710668,-74.011029,25,N..N20R,"[1, 5, N, R, W]"


In [75]:
stop_train2.to_pickle('static/data/GTFS_nyc_Subway/stop_train.pkl')

In [76]:
stop_train2.to_csv('static/data/GTFS_nyc_Subway/stop_train.csv', index=False)

In [38]:
route_id = transfer.merge(stop_train2, on='stop_name')
route_id.head()

,stop_name,lines_x,stop_id,train,stop_lat,stop_lon,stop_sequence,train+direction,lines_y
0,1 Av,[L],L06,L,40.730953,-73.981628,20,L..N01R,[L]
1,103 St,"[1, 4, 6, A, B, C]",119,1,40.799446,-73.968379,22,1..N03R,"[1, 4, 6, A, B, C]"
2,103 St,"[1, 4, 6, A, B, C]",624,4,40.790600,-73.947478,37,4..N13R,"[1, 4, 6, A, B, C]"
3,103 St,"[1, 4, 6, A, B, C]",624,6,40.790600,-73.947478,17,6..N01R,"[1, 4, 6, A, B, C]"
4,103 St,"[1, 4, 6, A, B, C]",A18,A,40.796092,-73.961454,46,A..N09R,"[1, 4, 6, A, B, C]"


In [58]:
route_id.columns = ['stop_name', 'line_num', 'stop_id', 'route_id', 'stop_lat', 'stop_lon', 'node_id']
route_id.head()

,stop_name,line_num,stop_id,route_id,stop_lat,stop_lon,node_id
0,1 Av,1,L06,L,40.730953,-73.981628,L06_L
1,103 St,6,119,1,40.799446,-73.968379,119_1
2,103 St,6,624,4,40.790600,-73.947478,624_4
3,103 St,6,624,6,40.790600,-73.947478,624_6
4,103 St,6,A18,A,40.796092,-73.961454,A18_A


In [59]:
route_id.to_csv('route_id.csv')

In [40]:
shape = pd.read_csv('static/data/GTFS_nyc_Subway/shapes.txt')
shape.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,1..N03R,40.702068,-74.013664,0,NaN
1,1..N03R,40.703199,-74.014792,1,NaN
2,1..N03R,40.703226,-74.014820,2,NaN
3,1..N03R,40.703253,-74.014846,3,NaN
4,1..N03R,40.703280,-74.014870,4,NaN


In [77]:
#get lines
shape['lines'], shape['route'] = zip(*shape['shape_id'].map(lambda x: x.split('.')))

In [81]:
del shape['route']

In [83]:
routes = pd.read_csv('static/data/GTFS_nyc_Subway/routes.txt')
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,MTA NYCT,1,Broadway - 7 Avenue Local,Trains operate between 242 St in the Bronx and...,1,http://web.mta.info/nyct/service/pdf/t1cur.pdf,EE352E,NaN
1,2,MTA NYCT,2,7 Avenue Express,"Trains operate between Wakefield-241 St, Bronx...",1,http://web.mta.info/nyct/service/pdf/t2cur.pdf,EE352E,
2,3,MTA NYCT,3,7 Avenue Express,"Trains operate between 148 St, 7 Av, Manhattan...",1,http://web.mta.info/nyct/service/pdf/t3cur.pdf,EE352E,NaN
3,4,MTA NYCT,4,Lexington Avenue Express,Trains operate daily between Woodlawn/Jerome A...,1,http://web.mta.info/nyct/service/pdf/t4cur.pdf,00933C,NaN
4,5,MTA NYCT,5,Lexington Avenue Express,"Weekdays daytime, most trains operate between ...",1,http://web.mta.info/nyct/service/pdf/t5cur.pdf,00933C,NaN


In [84]:
shape.shape, routes.shape

((122382, 6), (29, 9))

In [88]:
new_shape = shape.merge(routes, left_on='lines', right_on='route_id')

In [93]:
new_shape = new_shape[['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence','shape_dist_traveled', 'route_color']]

In [94]:
new_shape.to_csv('static/data/GTFS_nyc_Subway/new_shape.csv', index=False)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,route_color
0,1..N03R,40.702068,-74.013664,0,NaN,EE352E
1,1..N03R,40.703199,-74.014792,1,NaN,EE352E
2,1..N03R,40.703226,-74.014820,2,NaN,EE352E
3,1..N03R,40.703253,-74.014846,3,NaN,EE352E
4,1..N03R,40.703280,-74.014870,4,NaN,EE352E
5,1..N03R,40.703307,-74.014893,5,NaN,EE352E
6,1..N03R,40.703335,-74.014914,6,NaN,EE352E
7,1..N03R,40.703363,-74.014933,7,NaN,EE352E
8,1..N03R,40.703391,-74.014952,8,NaN,EE352E
9,1..N03R,40.703419,-74.014968,9,NaN,EE352E
